##**Import Libraries**

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
! pip install -q transformers[sentencepiece] fastai ohmeow-blurr nbdev

In [33]:
! pip install -q onnxruntime

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 493.6 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 2.8 MB/s eta 0:00:00


In [37]:
! pip install onnx onnxruntime-gpu onnxruntime_tools

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.7/15.7 MB 32.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 157.1/157.1 MB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.7/212.7 kB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 6.8 MB/s eta 0:00:00


In [4]:
import torch
from transformers import AutoModelForSequenceClassification, AutoConfig
from fastai.text.all import *
from blurr.text.data.all import *
from blurr.text.modeling.all import *

/usr/local/lib/python3.10/dist-packages/blurr/text/modeling/question_answering.py:31: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  squad_metric = load_metric("squad")
/usr/local/lib/python3.10/dist-packages/datasets/load.py:752: FutureWarning: The repository for squad contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.16.1/metrics/squad/squad.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


In [5]:
from tqdm.notebook import tqdm
import numpy as np

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
%cd /content/drive/MyDrive/Datasets/NLP_Project

/content/drive/MyDrive/Datasets/NLP_Project


##**Data**

In [8]:
df = pd.read_csv("movie_data.csv")
df.head()

,Title,Description,Genres
0,Rebel Moon: Part One - A Child of Fire,"When a peaceful settlement on the edge of a distant moon finds itself threatened by the armies of a tyrannical ruling force, a mysterious stranger living among its villagers becomes their best hope for survival.","Action, Adventure, Drama"
1,Saltburn,"A student at Oxford University finds himself drawn into the world of a charming and aristocratic classmate, who invites him to his eccentric family's sprawling estate for a summer never to be forgotten.","Comedy, Drama, Thriller"
2,Leave the World Behind,"A family's getaway to a luxurious rental home takes an ominous turn when a cyberattack knocks out their devices, and two strangers appear at their door.","Drama, Mystery, Thriller"
3,Wonka,"With dreams of opening a shop in a city renowned for its chocolate, a young and poor Willy Wonka discovers that the industry is run by a cartel of greedy chocolatiers.","Adventure, Comedy, Family"
4,Aquaman and the Lost Kingdom,"Black Manta seeks revenge on Aquaman for his father's death. Wielding the Black Trident's power, he becomes a formidable foe. To defend Atlantis, Aquaman forges an alliance with his imprisoned brother. They must protect the kingdom.","Action, Adventure, Fantasy"


In [9]:
df = df.dropna().reset_index(drop=True)
df = df.drop_duplicates()
df.shape

(12474, 3)

In [10]:
genres_list = df['Genres'].tolist()

cleaned_genres_list = []
for genres in genres_list:
    genre_list = [genre.strip().replace('-', '') for genre in genres.split(',')]
    genre_list = ['Sci-fi' if genre.lower() == 'sci' or genre.lower() == 'fi' else genre for genre in genre_list]
    cleaned_genres_list.append(', '.join(genre_list))

# Update the 'Genres' column in the DataFrame
df['genres'] = cleaned_genres_list

In [11]:
df.drop(columns=['Genres'], inplace=True)

**Data Check**

In [12]:
genre_count = {}
for genres in cleaned_genres_list:
    genre_list = [genre.strip() for genre in genres.split(',')]
    for genre in genre_list:
        if genre in genre_count:
            genre_count[genre] += 1
        else:
            genre_count[genre] = 1

print(f"Number of Genres: {len(genre_count)}")
print(genre_count)

Number of Genres: 23
{'Action': 2871, 'Adventure': 2066, 'Drama': 6657, 'Comedy': 4240, 'Thriller': 2177, 'Mystery': 1277, 'Family': 741, 'Fantasy': 946, 'Romance': 2072, 'Biography': 786, 'History': 426, 'Sport': 246, 'Crime': 2152, 'Musical': 193, 'Sci-fi': 1790, 'Horror': 1754, 'Animation': 548, 'Music': 338, 'War': 252, 'Documentary': 179, 'Western': 171, 'Film': 51, 'Noir': 51}


In [13]:
threshold = int(len(df) * 0.01)
rare_genres = [key for key, value in genre_count.items() if value < threshold]
len(rare_genres), rare_genres[:5]

(2, ['Film', 'Noir'])

In [14]:
genres_list = df.genres.to_list()
revised_genre_list = []
indices_to_drop = []

for idx, genres in enumerate(genres_list):
  genre_list = [genre.strip() for genre in genres.split(',')]
  revised_genres = []

  for genre in genre_list:
    if genre not in rare_genres:
      revised_genres.append(genre)

  if len(revised_genres) == 0:
    indices_to_drop.append(idx)
  else:
    revised_genre_list.append(revised_genres)

df = df.drop(indices_to_drop).reset_index(drop=True)
df.shape

(12474, 3)

In [15]:
df['revised_genres'] = revised_genre_list

In [16]:
revised_genres_list = df.revised_genres.to_list()
revised_genre_count = {}
for genres in revised_genres_list:
  genre_list = genres
  for genre in genre_list:
    if genre in revised_genre_count.keys():
      revised_genre_count[genre] += 1
    else:
      revised_genre_count[genre] = 1
print(f"Number of Genres: {len(revised_genre_count)}")
print(revised_genre_count)

Number of Genres: 21
{'Action': 2871, 'Adventure': 2066, 'Drama': 6657, 'Comedy': 4240, 'Thriller': 2177, 'Mystery': 1277, 'Family': 741, 'Fantasy': 946, 'Romance': 2072, 'Biography': 786, 'History': 426, 'Sport': 246, 'Crime': 2152, 'Musical': 193, 'Sci-fi': 1790, 'Horror': 1754, 'Animation': 548, 'Music': 338, 'War': 252, 'Documentary': 179, 'Western': 171}


In [17]:
encode_genre_types = { key: idx for idx, (key, value) in enumerate(revised_genre_count.items())}
with open("genre_types_encoded.json", "w") as fp:
  json.dump(encode_genre_types, fp)

In [18]:
labels = list(encode_genre_types.keys())
len(labels), labels[:5]

(21, ['Action', 'Adventure', 'Drama', 'Comedy', 'Thriller'])

In [19]:
# We need this because for multilabel classification all genres have possibility to be present in the predictions
categorical_genre_list = []
revised_genres_list = df.revised_genres.to_list()

for revised_genres in revised_genres_list:
  categorical_list = [0] * len(encode_genre_types)
  for genre in revised_genres:
    genre_type_index = encode_genre_types[genre]
    categorical_list[genre_type_index] = 1
  categorical_genre_list.append(categorical_list)

df['genre_cat_list'] = categorical_genre_list
categorical_genre_list[3][:23]

[0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

##**Data Split**

In [20]:
splitter = RandomSplitter(valid_pct=0.1, seed=42)
train_ids, valid_ids = splitter(df)
len(train_ids), len(valid_ids)

(11227, 1247)

In [21]:
valid_df = df.loc[valid_ids]
valid_df.head()

,Title,Description,genres,revised_genres,genre_cat_list
2976,Stepmom,"A terminally-ill woman must deal with her ex-husband's new lover, who will be their children's stepmother.","Comedy, Drama","[Comedy, Drama]","[0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
6682,Nowhere Boy,"A chronicle of John Lennon's first years, focused mainly in his adolescence and his relationship with his stern aunt Mimi, who raised him, and his absentee mother Julia, who re-entered his life at a crucial moment in his young life.","Biography, Drama, Music","[Biography, Drama, Music]","[0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]"
3966,Gangs of Wasseypur,"A clash between Sultan and Shahid Khan leads to the expulsion of Khan from Wasseypur, and ignites a deadly blood feud spanning three generations.","Action, Comedy, Crime","[Action, Comedy, Crime]","[1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]"
3463,Sex and Lucía,"Various lives converge on an isolated island, all connected by an author whose novel has become inextricably entwined with his own life.","Drama, Romance","[Drama, Romance]","[0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
3510,The Thundermans Return,"It centers on the family as they begin a new era of superhero crimefighting. When one save goes wrong, the Thundermans are sent back to Hiddenville.","Action, Comedy, Crime","[Action, Comedy, Crime]","[1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]"


##**Fastai and Blurr Inference**

In [22]:
model_path = "models/multi-movie_genre-classifier-stage-3.pkl"
learner_inf = load_learner(model_path)

In [25]:
learner_inf.blurr_predict("I have promises to keep")[0]['labels']

['Comedy', 'Family']

##**Evaluation**

In [26]:
from sklearn import metrics

def metric_measures(test_df, preds):

  targets = [np.asarray(target) for target in test_df['genre_cat_list'].to_list()]
  outputs = [np.asarray(pred) for pred in preds]


  accuracy = metrics.accuracy_score(targets, outputs)
  f1_score_micro = metrics.f1_score(targets, outputs, average='micro')
  f1_score_macro = metrics.f1_score(targets, outputs, average='macro')

  print(f"F1 Score (Micro) = {f1_score_micro}")
  print(f"F1 Score (Macro) = {f1_score_macro}")

  return

In [27]:
preds = []
for idx, row in tqdm(valid_df.iterrows(), total=len(valid_df)):
  desc = row['Description']
  labels = learner_inf.blurr_predict(desc)[0]['labels']
  pred_genres = [0] * len(encode_genre_types)
  for label in labels:
    pred_genres[encode_genre_types[label]] = 1
  preds.append(pred_genres)

preds[0][:20]

  0%|          | 0/1247 [00:00<?, ?it/s]

[0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [28]:
metric_measures(valid_df, preds)

F1 Score (Micro) = 0.5687676700482288
F1 Score (Macro) = 0.45357224732258555


##**Convert to ONNX**

In [29]:
model_path = "models/multi-movie_genre-classifier-stage-3.pkl"
learner_inf = load_learner(model_path)

In [32]:
classifier = learner_inf.model.hf_model.eval()

torch.onnx.export(
    classifier,
    torch.LongTensor([[0] * 512]),
    'models/movie_genre-classifier.onnx',
    input_names=['input_ids'],
    output_names=['output'],
    opset_version=13,
    dynamic_axes={
        'input_ids': {0: 'batch_size', 1: 'sequence_len'},
        'output': {0: 'batch_size'}
    }
)

In [38]:
from onnxruntime.quantization import quantize_dynamic, QuantType

onnx_model_path = 'models/movie_genre-classifier.onnx'
quantized_onnx_model_path = 'models/movie_gnere-classifier-quantized.onnx'

quantize_dynamic(
    onnx_model_path,
    quantized_onnx_model_path,
    weight_type=QuantType.QUInt8,
)

Ignore MatMul due to non constant B: /[/roberta/encoder/layer.0/attention/self/MatMul]
Ignore MatMul due to non constant B: /[/roberta/encoder/layer.0/attention/self/MatMul_1]
Ignore MatMul due to non constant B: /[/roberta/encoder/layer.1/attention/self/MatMul]
Ignore MatMul due to non constant B: /[/roberta/encoder/layer.1/attention/self/MatMul_1]
Ignore MatMul due to non constant B: /[/roberta/encoder/layer.2/attention/self/MatMul]
Ignore MatMul due to non constant B: /[/roberta/encoder/layer.2/attention/self/MatMul_1]
Ignore MatMul due to non constant B: /[/roberta/encoder/layer.3/attention/self/MatMul]
Ignore MatMul due to non constant B: /[/roberta/encoder/layer.3/attention/self/MatMul_1]
Ignore MatMul due to non constant B: /[/roberta/encoder/layer.4/attention/self/MatMul]
Ignore MatMul due to non constant B: /[/roberta/encoder/layer.4/attention/self/MatMul_1]
Ignore MatMul due to non constant B: /[/roberta/encoder/layer.5/attention/self/MatMul]
Ignore MatMul due to non constant

## **ONNX Inference**


### **Normal ONNX**

In [39]:
import onnxruntime as rt
from transformers import AutoTokenizer
import torch

tokenizer = AutoTokenizer.from_pretrained("distilroberta-base")

class_labels = list(encode_genre_types.keys())

inf_session = rt.InferenceSession('models/movie_genre-classifier.onnx')
input_name = inf_session.get_inputs()[0].name
output_name = inf_session.get_outputs()[0].name

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [40]:
preds = []
for idx, row in tqdm(valid_df.iterrows(), total=valid_df.shape[0]):
  desc = row['Description']
  input_ids = tokenizer(desc)['input_ids'][:512]

  probs = inf_session.run([output_name], {input_name: [input_ids]})[0]
  probs = torch.FloatTensor(probs)

  masks = torch.sigmoid(probs) >= 0.5
  labels = [class_labels[idx] for idx, mask in enumerate(masks[0]) if mask]

  pred_genres = [0] * len(encode_genre_types)
  for label in labels:
    pred_genres[encode_genre_types[label]] = 1
  preds.append(pred_genres)

  0%|          | 0/1247 [00:00<?, ?it/s]

In [41]:
metric_measures(valid_df, preds)

F1 Score (Micro) = 0.5669709543568465
F1 Score (Macro) = 0.4527090905285143


###**Quantized ONNX**

In [42]:
import onnxruntime as rt
from transformers import AutoTokenizer
import torch

tokenizer = AutoTokenizer.from_pretrained("distilroberta-base")

class_labels = list(encode_genre_types.keys())

inf_session = rt.InferenceSession('models/movie_gnere-classifier-quantized.onnx')
input_name = inf_session.get_inputs()[0].name
output_name = inf_session.get_outputs()[0].name

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [43]:
preds = []
for idx, row in tqdm(valid_df.iterrows(), total=valid_df.shape[0]):
  desc = row['Description']
  input_ids = tokenizer(desc)['input_ids'][:512]

  probs = inf_session.run([output_name], {input_name: [input_ids]})[0]
  probs = torch.FloatTensor(probs)

  masks = torch.sigmoid(probs) >= 0.5
  labels = [class_labels[idx] for idx, mask in enumerate(masks[0]) if mask]

  pred_genres = [0] * len(encode_genre_types)
  for label in labels:
    pred_genres[encode_genre_types[label]] = 1
  preds.append(pred_genres)

  0%|          | 0/1247 [00:00<?, ?it/s]

In [44]:
metric_measures(valid_df, preds)

F1 Score (Micro) = 0.5652173913043478
F1 Score (Macro) = 0.4559900655535849
